# From Scratch to Production: Oceananigans.jl

In the previous notebooks we built a 2D incompressible solver by hand: staggered C-grid, finite differences, FFT pressure solver, projection method, multiple dispatch for advection schemes

Now we'll see how **Oceananigans.jl** — a production ocean modeling package — solves the same equations and much more. Under the hood, Oceananigans uses the exact same ingredients:
- **Arakawa C-grid** (staggered grid) → `RectilinearGrid`
- **Projection method** with pressure Poisson solver → `NonhydrostaticModel`
- **KernelAbstractions.jl** for CPU/GPU portability → just pass `GPU()` instead of `CPU()`
- **Multiple dispatch** for extensible physics → advection schemes, closures, buoyancy models

But it adds capabilities we didn't build: 3D, boundary conditions, tracer transport, buoyancy, adaptive time stepping, output management, and more.

# New physics: Gravity

The ocean is a **stratified** fluid: it experiences the force of gravity.
How do we represent this in out Navier-Stokes equations?
We employ a powerful assumption: the **Boussinesq** approximation. The equations are still incompressible (or-quasi), just that the w-equation has an extra gravity term.

$$\frac{\partial w}{\partial t} = - (\boldsymbol{u} \cdot \boldsymbol{\nabla})w - \frac{\partial p}{\partial z} + g\frac{\rho - \rho_0}{\rho_0} + \nu \nabla^2 w$$

It works only when the changes in density are tiny!
We have a new variable $\rho$, so we need extra equations. $\rho$ depends on temperature and salinity, so we need to add equations that solve for $T$ and $S$

$$\frac{\partial T}{\partial t} =  - (\boldsymbol{u} \cdot \boldsymbol{\nabla})T + \kappa \nabla^2 T$$
$$\frac{\partial S}{\partial t} =  - (\boldsymbol{u} \cdot \boldsymbol{\nabla})S + \kappa \nabla^2 S$$

## Cabbeling: When Mixing Creates Density

In this notebook we simulate **cabbeling** — convection driven by the nonlinearity of the equation of state of water.
We start with a stable fluid: hot water (7.55 °C) above cold water (1 °C), both at the **same density**.

<img src="images/water_eos.png" width="400">

The fluid is initially at rest, but when mixing occurs at the interface, the resulting intermediate temperatures
correspond to *higher* density (fresh water is densest near 4 °C). The mixed water sinks, driving convection from an initially stable state.

In [ ]:
using Pkg
Pkg.activate("./")

using Oceananigans
using Oceananigans.Models: seawater_density
using SeawaterPolynomials: TEOS10EquationOfState
using CairoMakie

## Step 1: Choose the Architecture

Remember how our hand-built solver used `get_backend(A)` to detect CPU vs GPU?
In Oceananigans the concept is basically the same.
To run on a GPU, just replace `CPU()` with `GPU()`. <br>
Everything — grids, fields, kernels — automatically adapts.

In [ ]:
arch = CPU()

## Step 2: Define the Grid

In our hand-built solver, we created `Grid(Nx, Ny, Lx, Ly)` and manually allocated staggered arrays.
Oceananigans' `RectilinearGrid` does all of this automatically — it sets up the Arakawa C-grid with
the correct staggering for all the necessary variables

In our solver yesterday we only had a `Periodic` domain. 
In Oceananigans we can use `Periodic`, `Bounded` or `Flat`.
Here we use `Bounded` walls in x and z (a vertical slice), with `Flat` in y (making it 2D in the x-z plane).
While yesterday we hardcoded index login in the operators, in Oceananigans we use "halo" cells to represent boundary conditions

In [ ]:
Nx, Ny = 300, 300
grid = RectilinearGrid(arch,
                       size = (Nx, Ny),
                       x = (0, 0.5),
                       z = (-0.5, 0.0),
                       halo = (5, 5), 
                       topology = (Bounded, Flat, Bounded))

## Step 3: Buoyancy — New Physics Beyond Our Solver

Our hand-built solver only had velocity and pressure. Oceananigans can couple the momentum equation to tracer dynamics
(in this case **buoyancy**) through an equation of state.
The equation of state tells you how density depends on temperature and salinity

$$\rho = EOS(T, S, p)$$

Here we use the **TEOS-10** equation of state — the international standard for seawater thermodynamics.
This is a nonlinear function $\rho(T, S, p)$ that relates density to temperature, salinity, and pressure.
This nonlinearity is exactly what drives cabbeling: even though $T_1 = 1°$C and $T_2 = 7.55°$C have the
same density, their average $T \approx 4°$C is **denser** (fresh water has maximum density near 4°C).

In [ ]:
equation_of_state = TEOS10EquationOfState(reference_density=1000)
buoyancy = SeawaterBuoyancy(; equation_of_state,
                              constant_salinity=0, 
                              gravitational_acceleration=9.80655)

## Step 4: Build the Model

This is the equivalent of our `NavierStokesModel`. Compare:

| Our solver | Oceananigans |
|------------|-------------|
| `NavierStokesModel(grid, ν; advection=Centered())` | `NonhydrostaticModel(grid; advection=WENO(), ...)` |
| `FFTPoissonSolver` for pressure | Built-in pressure solver (auto-selected) |
| `Centered()`, `Upwind()` dispatch | `CenteredSecondOrder()`, `WENO(order=7)`, ... |
| Forward Euler | `RungeKutta3` (default) |
| Explicit `ν` parameter | `closure = ScalarDiffusivity(ν=...)` |

Here we use **WENO(order=7)** — a high-order Weighted Essentially Non-Oscillatory scheme.
Remember how we used multiple dispatch to swap between `Centered` and `Upwind`?
Oceananigans does the same: WENO is just another advection type that dispatches to different reconstruction methods.

With high-order WENO providing implicit numerical dissipation, we don't need an explicit viscosity closure.

In [ ]:
model = NonhydrostaticModel(grid;
                            buoyancy,
                            advection=WENO(order=7),
                            tracers=:T)

## Step 5: Initial Conditions

In our hand-built solver, we initialized velocity fields directly by writing into the arrays.
Oceananigans provides a convenient `set!` function that accepts either arrays or functions of coordinates.

We initialize:
- **Temperature**: hot water ($T_2 = 7.55°$C) and cold water ($T_1 = 1°$C) arranged in a pattern
- **Velocity**: tiny random perturbations to trigger the instability

For fun, we'll initialize the temperature pattern from a logo image!

In [ ]:
T₁, T₂ = 1, 7.55 # ᵒC
Ξᵢ = (x, z) -> 1e-4 * randn()

using Colors: red

function deep_dive_mask(Nx, Ny)
    scale = 4
    W, H  = Nx * scale, Ny * scale
    scene = Scene(camera=campixel!, size=(W, H), backgroundcolor=:black)
    fsize = min(W, H) ÷ 4
    text!(scene, Point2f(W / 2, H / 2), text="Deep\nDive",
          fontsize=fsize, align=(:center, :center),
          color=:white, font=:bold, lineheight=1.1)
    img = colorbuffer(scene)
    Himg, Wimg = size(img)
    mask = zeros(Float32, Nx, Ny)
    for j in 1:Ny, i in 1:Nx
        ix = clamp(round(Int, (i - 0.5) / Nx * Wimg), 1, Wimg)
        iy = clamp(round(Int, (1 - (j - 0.5) / Ny) * Himg), 1, Himg)
        mask[i, j] = Float32(red(img[iy, ix]))
    end
    return mask
end

Nx, _, Nz = size(grid)

mask = deep_dive_mask(Nx, Nz)
Tᵢ = [mask[i, j] > 0 ? T₁ : T₂ for i in 1:Nx, j in 1:Nz]

set!(model, T=Tᵢ, u=Ξᵢ, w=Ξᵢ)

heatmap(interior(model.tracers.T, :, 1, :))

## Step 6: Simulation

In our solver, we called `run!(model, Δt, Tfinal)` in a simple loop.
Oceananigans wraps this in a `Simulation` object that manages time stepping, callbacks, and output.

In [ ]:
simulation = Simulation(model; Δt=0.1, stop_time=100)

### Progress callback

Oceananigans supports **callbacks** — functions that run during the simulation.
This is much more flexible than hardcoding print statements in a time loop.

In [ ]:
wall_time = [time_ns()]

function progress(sim)
    u, v, w = sim.model.velocities
    step_time = (time_ns() - wall_time[1]) * 1e-9
    @info string("Iter: ", iteration(sim), ", wall_time: ", prettytime(step_time), " time: ", prettytime(sim), ", max(w): ", maximum(abs, w))
    wall_time[1] = time_ns()
end

add_callback!(simulation, progress, IterationInterval(100))

### Adaptive time stepping

Our solver used a fixed $\Delta t$. Oceananigans can automatically adjust the time step
to satisfy a CFL condition — crucial for flows where velocity magnitudes change over time.

In [ ]:
conjure_time_step_wizard!(simulation, cfl=0.7)

### Output writers

In our solver we collected snapshots in a Julia array.
Oceananigans provides `OutputWriter`s that save fields to disk (JLD2, NetCDF) on a schedule.
This is essential for large simulations where you can't keep everything in memory.

Note how `seawater_density(model)` computes $\rho$ from $T$ via the equation of state — this is a
**computed field** (a lazy operation that evaluates on demand).

In [ ]:
ρ = seawater_density(model)
T = model.tracers.T

output_writer = JLD2Writer(model, (; ρ, T),
                           filename = "cabbeling",
                           schedule = TimeInterval(1),
                           overwrite_existing = true)

simulation.output_writers[:jld2] = output_writer

## Step 7: Run!

Just like `run!(model, Δt, Tfinal)` in our solver, but with all the bells and whistles:
adaptive time stepping, callbacks, output writing — all handled automatically.

In [ ]:
run!(simulation)

## Visualizing the Results

Oceananigans saves output as `FieldTimeSeries` objects that can be loaded from disk.
We use CairoMakie to create an animation of temperature and density evolution.

In [ ]:
using CairoMakie

ρt = FieldTimeSeries("cabbeling.jld2", "ρ")
Tt = FieldTimeSeries("cabbeling.jld2", "T")

Nt = length(ρt)
Nx = size(ρt, 1)

i = Int(Nx / 2)
n = Observable(length(ρt.times))
ρ = @lift interior(ρt[$n], :, 1, :)
T = @lift interior(Tt[$n], :, 1, :)
x, y, z = nodes(ρt)

set_theme!(Theme(fontsize=12))
fig = Figure(size=(1000, 400))

ρrange = (minimum(ρt[1]), maximum(ρt))

axT = Axis(fig[1, 2], xlabel="x (m)", ylabel="z (m)")
xlims!(axT,  0, 0.5)
ylims!(axT, -0.5, 0)

axρ = Axis(fig[1, 3], xlabel="x (m)", ylabel="z (m)")
xlims!(axρ,  0, 0.5)
ylims!(axρ, -0.5, 0)

hm = heatmap!(axT, x, z, T, colormap=:magma, colorrange=(1.55, 7))
Colorbar(fig[1, 1], hm, label="Temperature (ᵒC)", flipaxis=false)

hm = heatmap!(axρ, x, z, ρ, colormap=Makie.Reverse(:grays), colorrange=ρrange)
Colorbar(fig[1, 4], hm, label="Density (kg m⁻³)")

record(fig, "cabbeling_2d.mp4", 1:Nt, framerate=5) do nn
    mod(nn, 10) == 0 && @info "Drawing frame $nn of $Nt..."
    n[] = nn
end

In [ ]:
using Base64
function embed_mp4(path, width=900)
    data = base64encode(open(path))
    HTML("""<video controls autoplay loop width="$width"><source src="data:video/mp4;base64,$data" type="video/mp4"></video>""")
end

embed_mp4("cabbeling_2d.mp4")

## What We've Seen

Starting from raw GPU kernels and building a Navier-Stokes solver by hand,
we've now seen how Oceananigans wraps the same core ideas into a production tool:

| Concept | Our Solver | Oceananigans |
|---------|-----------|--------------|
| Grid | `Grid(Nx, Ny, Lx, Ly)` | `RectilinearGrid(arch; size, x, y, topology)` |
| Staggering | Manual `u[Nx, Ny]`, `v[Nx, Ny]` | Automatic C-grid with halo regions |
| Pressure | `FFTPoissonSolver` + `solve!` | Built-in (auto-selected for topology) |
| Advection | `Centered()`, `Upwind()` via dispatch | `WENO()`, `CenteredSecondOrder()`, ... |
| Time stepping | Forward Euler | `RungeKutta3` (default), `QuasiAdamsBashforth2` |
| Viscosity | Explicit `ν` parameter | `closure = ScalarDiffusivity(ν=...)` or LES closures |
| GPU support | `CuArray` + KernelAbstractions | Just pass `GPU()` as architecture |
| **New: Buoyancy** | — | Equation of state coupling |
| **New: Tracers** | — | `tracers = :T` (or `:S`, or any name) |
| **New: Output** | In-memory arrays | `JLD2OutputWriter`, `NetCDFOutputWriter` |
| **New: Adaptive Δt** | — | `conjure_time_step_wizard!(sim, cfl=0.7)` |

The key takeaway: **understanding the fundamentals** (staggered grids, projection method, GPU kernels)
gives you the foundation to use — and contribute to — production-grade tools like Oceananigans.

## Exercise: Ocean Salinity

In this simulation we used `constant_salinity = 0` (fresh water).
Real ocean water has salinity around 35 g/kg.

What do you think will happen if we increase the salinity to match ocean conditions?
Will cabbeling be stronger or weaker? Will it still occur at all?

> ...

### Step 1: Explore the equation of state

Use `SeawaterPolynomials` to compute the density at different temperatures for both
fresh water (S = 0) and seawater (S = 35). Plot $\rho(T)$ for both cases.

**Hint:** the temperature of maximum density for fresh water is ~4 °C.
What happens to this maximum at S = 35?

In [ ]:
# Step 1: Plot ρ(T) for S=0 and S=35
# Hint: you can use Oceananigans.Models.seawater_density or 
# SeawaterPolynomials.TEOS10.ρ to compute density


### Step 2: Run the simulation with ocean salinity

Modify the simulation above to use `constant_salinity = 35`.
You'll need to find two temperatures $T_1$ and $T_2$ that have **equal density** at $S = 35$.

**Hints:**
- The density maximum disappears at high salinity — seawater density increases monotonically as temperature decreases
- Try choosing $T_1$ and $T_2$ symmetric around some temperature and check if $\rho(T_1, S=35) \approx \rho(T_2, S=35)$
- Does cabbeling still occur? Is it stronger or weaker than in fresh water?

In [ ]:
# Step 2: Re-run the cabbeling simulation with constant_salinity = 35
# Copy and modify the setup from above — you only need to change:
#   1. The buoyancy (constant_salinity = 35)
#   2. The initial temperatures T₁ and T₂ (find equal-density pair at S = 35)


# The Hydrostatic approximation of the Navier Stokes equations

Solving the poisson equation for pressure might become a bit computationally expensive when we try to implement it in a GCM because
- we cannot use an FFT solver (only uniform grid cells in non-spherical coordinates)
- it is a heavy 3D computation

For this reason we simplify the Navier Stokes equations using the **Hydrostatic** approximation
The resulting equations are slightly simplified, but include some physics that is absent in the NonhydrostaticModel, the free surface!
Let's see what happens when we force the ocean with a tide

$$\frac{\partial \mathbf{u}}{\partial t} + (\mathbf{u} \cdot \nabla)\mathbf{u} + w\frac{\partial \mathbf{u}}{\partial z} + f\hat{z} \times \mathbf{u}= -\nabla p - g\nabla \eta + \nu \nabla^2 \mathbf{u}$$
$$\frac{\partial p}{\partial z} = b $$
$$\frac{\partial w}{\partial z} = - \nabla \cdot \mathbf{u}$$
$$\frac{\partial b}{\partial t} = - (\mathbf{u} \cdot \nabla)b - w\frac{\partial b}{\partial z} + \kappa \nabla^2 b$$

Here $ \mathbf{u}$ and $\mathbf{\nabla}$ are the horizontal velocity ($u$, $v$) and the horizontal gradient operator ($\partial_x$, $\partial_y$)




In [ ]:
using Oceananigans
using Oceananigans.Units
using CairoMakie

### The Grid: A 2D Ocean Slice with a Seamount

We create a 2000 km periodic channel, 2 km deep, with a Gaussian seamount in the middle.
Oceananigans uses `ImmersedBoundaryGrid` to carve out the topography — cells inside the mountain are masked out.

In [ ]:
Nx, Nz = 256, 128
H = 2kilometers
L = 1000kilometers

underlying_grid = RectilinearGrid(arch,
                                  size = (Nx, Nz), 
                                  halo = (4, 4),
                                  x = (-L, L), 
                                  z = (-H, 0),
                                  topology = (Periodic, Flat, Bounded))

# A Gaussian seamount: 250m tall, 20km wide
h₀ = 250meters
width = 20kilometers
hill(x) = h₀ * exp(-x^2 / 2width^2)
bottom(x) = -H + hill(x)

grid = ImmersedBoundaryGrid(underlying_grid, GridFittedBottom(bottom))

In [ ]:
# Let's visualize our domain
x = xnodes(grid, Center())
bottom_boundary = interior(grid.immersed_boundary.bottom_height, :, 1, 1)

fig = Figure(size = (700, 200))
ax = Axis(fig[1, 1], xlabel="x [km]", ylabel="z [m]",
          limits=((-L/1e3, L/1e3), (-H, 0)))
band!(ax, x / 1e3, bottom_boundary, zeros(length(x)), color = :mediumblue)
fig

### Tidal Forcing

The Moon's gravity drives a barotropic (depth-uniform) tide. We model the lunar $M_2$ tide as a sinusoidal body force on $u$:

$$F_u = A_2 \sin(\omega_2 t)$$

where $\omega_2 = 2\pi / T_2$ and $T_2 = 12.421$ hours is the $M_2$ tidal period.

The **excursion parameter** $\epsilon = U_{\text{tidal}} / (\omega_2 \sigma)$ measures how far the tide pushes fluid relative to the seamount width. We pick $\epsilon = 0.1$ (a gentle tide).

In [ ]:
# Coriolis: mid-latitudes f-plane
coriolis = FPlane(latitude = -45)

# M₂ tidal parameters
T₂ = 12.421hours
ω₂ = 2π / T₂
ϵ  = 0.1                                     # excursion parameter
U₂ = ϵ * ω₂ * width                          # tidal velocity amplitude
A₂ = U₂ * (ω₂^2 - coriolis.f^2) / ω₂        # forcing amplitude

@inline tidal_forcing(x, z, t, p) = p.A₂ * sin(p.ω₂ * t)
u_forcing = Forcing(tidal_forcing, parameters=(; A₂, ω₂))

### The Hydrostatic Model

Here's our new model! Notice: `HydrostaticFreeSurfaceModel` instead of `NonhydrostaticModel`.

We use `BuoyancyTracer()` — a simple buoyancy variable $b$ (no equation of state needed here).
The ocean is linearly stratified: $b(z) = N^2 z$, where $N^2 = 10^{-4}$ s$^{-2}$ is the buoyancy frequency.

In [ ]:
model = HydrostaticFreeSurfaceModel(grid; coriolis,
                                    buoyancy = BuoyancyTracer(),
                                    tracers = :b,
                                    momentum_advection = WENO(),
                                    tracer_advection = WENO(),
                                    forcing = (; u = u_forcing))

# Initial conditions: tidal flow + linear stratification
Nᵢ² = 1e-4  # buoyancy frequency squared [s⁻²]
bᵢ(x, z) = Nᵢ² * z
set!(model, u=U₂, b=bᵢ)

### Simulation Setup & Output

We run for 4 days (about 8 tidal cycles) and save snapshots every 30 minutes.
We also save $u' = u - \bar{u}$ (the deviation from the domain-averaged flow) to see the internal waves more clearly.

In [ ]:
using Printf

simulation = Simulation(model; Δt=5minutes, stop_time=4days)

# Progress callback
wall_clock = Ref(time_ns())
function progress(sim)
    elapsed = 1e-9 * (time_ns() - wall_clock[])
    @info @sprintf("Iter: %d, time: %s, wall time: %s, max|w|: %.3e m/s",
                   iteration(sim), prettytime(sim), prettytime(elapsed),
                   maximum(abs, sim.model.velocities.w))
    wall_clock[] = time_ns()
end
add_callback!(simulation, progress, IterationInterval(200))

# Output: u' (perturbation velocity), w, b, and N²
using Printf
u, v, w = model.velocities
b = model.tracers.b
U = Field(Average(u))
u′ = u - U
N² = ∂z(b)

simulation.output_writers[:fields] = JLD2Writer(model, (; u, u′, w, b, N²);
                                                filename = "internal_tide",
                                                schedule = TimeInterval(30minutes),
                                                overwrite_existing = true)

In [ ]:
run!(simulation)

### Visualizing Internal Tides

Let's animate the results! We'll show three panels:
- **$u'$**: perturbation velocity — the internal wave signal with the barotropic tide removed
- **$w$**: vertical velocity — shows where fluid is moving up and down
- **$N^2$**: stratification — watch how the internal waves perturb the density layers

In [ ]:
u′_t = FieldTimeSeries("internal_tide.jld2", "u′")
 w_t = FieldTimeSeries("internal_tide.jld2", "w")
N²_t = FieldTimeSeries("internal_tide.jld2", "N²")

times = u′_t.times
umax = maximum(abs, u′_t[end])
wmax = maximum(abs, w_t[end])

n = Observable(1)
title = @lift @sprintf("t = %.2f days (%.1f tidal cycles)", times[$n] / day, times[$n] / T₂)

u′ₙ = @lift interior(u′_t[$n], :, 1, :)
 wₙ = @lift interior( w_t[$n], :, 1, :)
N²ₙ = @lift interior(N²_t[$n], :, 1, :)

fig = Figure(size = (800, 900))
fig[1, :] = Label(fig, title, fontsize=20, tellwidth=false)

ax1 = Axis(fig[2, 1]; title="u' perturbation velocity")
hm1 = heatmap!(ax1, u′ₙ; nan_color=:gray, colorrange=(-umax, umax), colormap=:balance)
Colorbar(fig[2, 2], hm1, label="m/s")

ax2 = Axis(fig[3, 1]; title="w vertical velocity")
hm2 = heatmap!(ax2, wₙ; nan_color=:gray, colorrange=(-wmax, wmax), colormap=:balance)
Colorbar(fig[3, 2], hm2, label="m/s")

ax3 = Axis(fig[4, 1]; title="N² stratification")
hm3 = heatmap!(ax3, N²ₙ; nan_color=:gray, colorrange=(0.9Nᵢ², 1.1Nᵢ²), colormap=:magma)
Colorbar(fig[4, 2], hm3, label="s⁻²")

In [ ]:
record(fig, "internal_tide.mp4", 1:length(times), framerate=16) do i
    mod(i, 20) == 0 && @info "Frame $i / $(length(times))"
    n[] = i
end

### What's Happening?

Watch the animation closely:
1. **The tide sloshes back and forth** over the seamount (the barotropic forcing)
2. **Internal wave beams** radiate away from the seamount at an angle — these are the internal tides!
3. The beam angle depends on the ratio of tidal frequency to buoyancy frequency: $\theta = \cos^{-1}(\omega_2 / N)$
4. **Stratification ($N^2$)** gets perturbed as the waves pass — density surfaces oscillate up and down